In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
%matplotlib inline 

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars xgboost4j-spark-0.72.jar,xgboost4j-0.72.jar pyspark-shell'


In [3]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, GBTClassifier,OneVsRest, OneVsRestModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
spark = SparkSession.builder.master("local[*]").appName("Data cleaning").getOrCreate()

In [4]:
#spark.sparkContext.addPyFile("/home/ml/Documents/crimes_chigaco_spark/notebooks/sparkxgb.zip")

In [5]:
#from sparkxgb import XGBoostEstimator, XGBoostClassificationModel

In [6]:
from src.data.make_dataset import LoadDataframe
from src.features.build_features import extract_features_classification

In [7]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    

config_file = "/home/ml/Documents/Crime_Chicago_Spark/config/config.yml"
config = _build_configuration(config_file)

In [8]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')


CPU times: user 584 µs, sys: 52 µs, total: 636 µs
Wall time: 645 µs


In [9]:
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()
df_socio = obj_df_loaded.df_socio()
df_crimes_socio = obj_df_loaded.df_crime_socio()
obj_extract_features_classification = extract_features_classification(config, df_crimes_socio, df_temp, df_sky)

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [10]:
from pyspark.sql.functions import isnan, when, count, col, isnull

In [11]:
df_ml = obj_extract_features_classification.extract_feature()

In [12]:
df_ml.limit(5).toPandas().T

,0,1,2,3,4
year,2013,2013,2013,2013,2013
month,11,12,6,2,2
day,7,2,10,7,19
hour,8,4,10,15,15
community_area_number,43,58,44,54,54
id,10754746,9412469,9166760,9001193,9015161
cas_number,HZ517281,HW555076,HW312239,HW147978,HW162837
date,2013-11-07 08:00:00,2013-12-02 04:28:00,2013-06-10 10:20:00,2013-02-07 15:45:00,2013-02-19 15:45:00
block,019XX E 75TH ST,028XX W 48TH PL,080XX S COTTAGE GROVE AVE,130XX S LANGLEY AVE,130XX S LANGLEY AVE
iucr,0890,0560,0486,0460,0460


In [13]:
df_ml.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- community_area_number: string (nullable = true)
 |-- id: string (nullable = true)
 |-- cas_number: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- block: string (nullable = true)
 |-- iucr: string (nullable = true)
 |-- primary_type: string (nullable = true)
 |-- description: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- arrest: string (nullable = true)
 |-- domestic: string (nullable = true)
 |-- beat: string (nullable = true)
 |-- district: string (nullable = true)
 |-- ward: string (nullable = true)
 |-- fbi_code: string (nullable = true)
 |-- x_coordinate: string (nullable = true)
 |-- y_coordinate: string (nullable = true)
 |-- updated_on: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- location: string (nulla

In [ ]:
#df_ml.select([count(when(isnan(c), c)).alias(c) for c in df_ml.columns]).toPandas()

In [14]:
categoricalColumns = ['domestic']
numericCols = ['year', 'month', 'day', 'hour','minute', 'latitude',
               'longitude', 'isStreet', 'isAV',
               'isBLVD', 'isRD', 'isPL', 'isBROADWAY',
               'isPKWY', 'duree_day', 'minute','dayofmonth',
               'dayofyear','dayofweek','Temperature',
               'pct_housing_crowded',
               'pct_households_below_poverty',
               'pct_age16_unemployed',
               'pct_age25_no_highschool',
               'pct_not_working_age',
               'per_capita_income',
               'hardship_index',
                'Chicago_broken clouds',
                'Chicago_drizzle',
                 'Chicago_few clouds',
                'Chicago_fog',
                 'Chicago_haze',
                 'Chicago_heavy intensity drizzle',
                 'Chicago_heavy intensity rain',
                 'Chicago_heavy snow',
                 'Chicago_light intensity drizzle',
                 'Chicago_light rain',
                 'Chicago_light rain and snow',
                 'Chicago_light snow',
                 'Chicago_mist',
                 'Chicago_moderate rain',
                 'Chicago_overcast clouds',
                 'Chicago_proximity thunderstorm',
                 'Chicago_scattered clouds',
                 'Chicago_sky is clear',
                 'Chicago_snow',
                 'Chicago_thunderstorm',
                 'Chicago_thunderstorm with heavy rain',
                 'Chicago_thunderstorm with light rain',
                 'Chicago_thunderstorm with rain',
                'Chicago_very heavy rain'
]


In [15]:
rf = RandomForestClassifier(labelCol='label', featuresCol='features')


In [16]:
df_train, df_test = df_ml.randomSplit([0.7, 0.3])

In [17]:
%%time
stages = []
stringIndexer_label = StringIndexer(inputCol = 'primary_type', outputCol = 'label').fit(df_train)

CPU times: user 230 ms, sys: 46.5 ms, total: 276 ms
Wall time: 57min 41s


In [18]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

In [19]:
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol = categoricalCol, outputCol = categoricalCol + 'Index')
    encoder = OneHotEncoderEstimator(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages.append(stringIndexer)
    stages.append(encoder)
stages.append(stringIndexer_label)
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages.append(assembler)
stages.append(rf)
stages.append(labelConverter)

In [20]:
pipeline_and_model = Pipeline(stages = stages)

In [21]:
#paramGrid = ParamGridBuilder().addGrid(rf.numTrees, [100, 110 ,200]).addGrid(rf.maxDepth, [10,12,15]).build()
paramGrid = (ParamGridBuilder().addGrid(rf.numTrees, [ 50 ,60])
             .addGrid(rf.maxDepth, [5,8])
             .build())


In [22]:
crossval = CrossValidator(estimator=pipeline_and_model,
                          estimatorParamMaps=paramGrid,
                          evaluator= MulticlassClassificationEvaluator(),
                          numFolds=10) 

In [23]:
cvModel = crossval.fit(df_train)

In [24]:
bestPipeline = cvModel.bestModel

In [25]:
%%time
df_test_pred = bestPipeline.transform(df_test)

CPU times: user 33 ms, sys: 5.17 ms, total: 38.1 ms
Wall time: 272 ms


In [26]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [27]:
predictions = df_test_pred.select(col("label"), col("prediction"))

In [28]:
%%time
predictions_ = predictions.cache()

CPU times: user 1.73 ms, sys: 346 µs, total: 2.08 ms
Wall time: 125 ms


In [29]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [30]:
accuracy = evaluator.evaluate(predictions_)

In [31]:
print(accuracy)

0.6682717688652834


In [32]:
df_test_pred.limit(3).toPandas().T

,0,1,2
year,2013,2013,2013
month,6,3,5
day,10,13,1
hour,10,20,4
community_area_number,44,54,23
id,9166760,9045921,9110189
cas_number,HW312239,HW191139,HW255069
date,2013-06-10 10:20:00,2013-03-13 20:35:00,2013-05-01 04:10:00
block,080XX S COTTAGE GROVE AVE,130XX S LANGLEY AVE,010XX N PULASKI RD
iucr,0486,2027,0460


In [33]:
df_test_pred.select('label', 'prediction', 'predictedLabel', 'primary_type').show(5)

+-----+----------+--------------------+--------------------+
|label|prediction|      predictedLabel|        primary_type|
+-----+----------+--------------------+--------------------+
|  1.0|       1.0|     ASSAULT_BATTERY|     ASSAULT_BATTERY|
|  2.0|       1.0|     ASSAULT_BATTERY|           NARCOTICS|
|  1.0|       2.0|           NARCOTICS|     ASSAULT_BATTERY|
|  0.0|       0.0|THEFT_ROBBERY_BUR...|THEFT_ROBBERY_BUR...|
|  2.0|       0.0|THEFT_ROBBERY_BUR...|           NARCOTICS|
+-----+----------+--------------------+--------------------+
only showing top 5 rows



In [34]:
bestRFModel = bestPipeline.stages[4]

In [35]:
bestParams = bestRFModel.extractParamMap()

In [36]:
bestParams

{Param(parent='RandomForestClassifier_58a8d3c8c9d1', name='cacheNodeIds', doc='If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees.'): False,
 Param(parent='RandomForestClassifier_58a8d3c8c9d1', name='checkpointInterval', doc='set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext'): 10,
 Param(parent='RandomForestClassifier_58a8d3c8c9d1', name='featureSubsetStrategy', doc='The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].'): 'auto',
 Param(parent='RandomForestClassifier_58a8d3c8c9d1', name='featuresCol', doc='features column name'): 'features',
 Param(parent='RandomForestClassifier_58a8d3c8c9d1', 

In [37]:
stages

[StringIndexer_f6ff1ee91158,
 OneHotEncoderEstimator_526862df0681,
 StringIndexer_da2730e98d7c,
 VectorAssembler_d43e5a26f444,
 RandomForestClassifier_58a8d3c8c9d1,
 IndexToString_a54c767c293b]

In [38]:
cvModel.avgMetrics

[0.6419291072356116,
 0.6464532246180849,
 0.6432412722131753,
 0.6485185185690535]

In [39]:
cvModel.bestModel.stages

[StringIndexer_f6ff1ee91158,
 OneHotEncoderEstimator_526862df0681,
 StringIndexer_da2730e98d7c,
 VectorAssembler_d43e5a26f444,
 RandomForestClassificationModel (uid=RandomForestClassifier_58a8d3c8c9d1) with 60 trees,
 IndexToString_a54c767c293b]

In [1]:
from setuptools import setup